In [2]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('full_emoji.csv')

# Explore the dataset
print(df.head())  # Display the first few rows of the DataFrame
print(df.info())  # Get information about the DataFrame, such as column names and data types


   # emoji  unicode                             name  \
0  1     😀  U+1F600                    grinning face   
1  2     😃  U+1F603      grinning face with big eyes   
2  3     😄  U+1F604  grinning face with smiling eyes   
3  4     😁  U+1F601   beaming face with smiling eyes   
4  5     😆  U+1F606          grinning squinting face   

                                               Apple  \
0  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
1  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
2  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
3  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
4  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   

                                              Google  \
0  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
1  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
2  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
3  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
4  data:image/png;base64,iVBORw0KGgoAAAANSUhEU

In [17]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the text descriptions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['name'])
vocab_size = len(tokenizer.word_index) + 1

# Encode the emojis
emoji_to_index = {emoji: index for index, emoji in enumerate(df['emoji'])}

In [23]:
input_sequences = tokenizer.texts_to_sequences(df['name'])
max_sequence_length = max(len(seq) for seq in input_sequences)

X = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
y = [emoji_to_index[emoji] for emoji in df['emoji']]


In [25]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 100, input_length=max_sequence_length),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(len(emoji_to_index), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [32]:
import numpy as np

# Convert y to numpy array
y = np.array(y)

# Then, continue with fitting the model
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100
57/57 [==============================] - 1s 13ms/step - loss: 0.2256 - accuracy: 0.9813
Epoch 2/100
57/57 [==============================] - 1s 12ms/step - loss: 0.1788 - accuracy: 0.9901
Epoch 3/100
57/57 [==============================] - 1s 12ms/step - loss: 0.1593 - accuracy: 0.9939
Epoch 4/100
57/57 [==============================] - 1s 12ms/step - loss: 0.1417 - accuracy: 0.9950
Epoch 5/100
57/57 [==============================] - 1s 13ms/step - loss: 0.1336 - accuracy: 0.9956
Epoch 6/100
57/57 [==============================] - 1s 13ms/step - loss: 0.1271 - accuracy: 0.9939
Epoch 7/100
57/57 [==============================] - 1s 12ms/step - loss: 0.1228 - accuracy: 0.9956
Epoch 8/100
57/57 [==============================] - 1s 12ms/step - loss: 0.1186 - accuracy: 0.9967
Epoch 9/100
57/57 [==============================] - 1s 20ms/step - loss: 0.1145 - accuracy: 0.9956
Epoch 10/100
57/57 [==============================] - 1s 25ms/step - loss: 0.1110 - accuracy: 0.9961

In [43]:
def text_to_emoji(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length, padding='post')
    prediction = model.predict(padded_sequence)
    predicted_index = prediction.argmax(axis=1)[0]
    for emoji, index in emoji_to_index.items():
        if index == predicted_index:
            return emoji
    return "Emoji not found"

# Example usage
input_text = input("Enter text: ")
result = text_to_emoji(input_text)
print("Emoji:", result)


Enter text:  crying face


1/1 [==============================] - 0s 47ms/step
Emoji: 😢


In [30]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
